# What to classify ?

Lecture 3 : we saw the KNN algorithm to classify samples into classes. 

In our dataset, several electronic devices consume power. Each device is part of a category of devices :
-multimedia = [computer, 3D_printer, internet_router, laptop, phone_charger, printer, screen, tv, sound_system],  
-kitchen = [boiler, coffee, freezer, fridge, micro_wave_oven],  
-cooling = [air_conditioner, fan],  
-other = [air_purifier, dehumidifier, radiator, solar_panel, vacuum].

Each category might have similar features, which could permit us to classify an unknown device. For example, in the category "washing", we expect all devices to be consuming power mainly during off-peak time, about once a week.

1- Pretreating our data (removing all "other" devices, making sure all devices have the same length of measurements, start at the same hour, maybe making sure they all start the same day of the week (to compute calculations depending on the we etc...))  
2- Finding relevant components that could permit us to distinguish two classes and implement them : a function that creates an array with all the components
Try to plot each component for each class to see if we can already see some differences between the classes, see if the components are relevant
3- KNN on all the components (multiple dimensions KNN, more than  2 dim)
For the KNN : plot the accuracy, the graphs for different values of neighbors (see Lecture 3)

# 3- KNN and classification

We now have the components that can distinguish two classes of devices. We perform the KNN algorithm to classify the devices.